<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/Bi-LSTM-GRU-intra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Dense, Bidirectional, GRU
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/intravehicle.csv')

In [4]:
y_data = data.pop('4')
x_data = data

In [5]:
train_x, test_x, train_y, test_y = train_test_split(x_data, y_data, test_size=0.2, random_state=0, shuffle=True)

In [6]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [7]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [8]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [9]:
model = Sequential()
model.add(Bidirectional(LSTM(100, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, go_backwards=True), input_shape=(len(x_data.columns), 1)))
model.add(Dense(100, activation='ReLU'))
model.add(GRU(100, activation='tanh', recurrent_activation='sigmoid'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 4, 200)           81600     
 l)                                                              
                                                                 
 dense (Dense)               (None, 4, 100)            20100     
                                                                 
 gru (GRU)                   (None, 100)               60600     
                                                                 
 dense_1 (Dense)             (None, 4)                 404       
                                                                 
Total params: 162,704
Trainable params: 162,704
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(train_x, train_y, batch_size=32) 

414237/414237 [==============================] - 2706s 7ms/step - loss: 0.0331 - accuracy: 0.9889


In [11]:
pred = model.predict(test_x, verbose=1)
pred_y = np.argmax(pred, axis=1)

103560/103560 [==============================] - 319s 3ms/step


In [12]:
confusion_matrix(test_y, pred_y)

array([[ 117727,       0,       0,       0],
       [      0,   94861,    3341,       0],
       [  23114,     194, 2824602,       0],
       [      0,       0,       0,  250056]])

In [13]:
report = classification_report(test_y, pred_y, digits=5)
print(report)

              precision    recall  f1-score   support

           0    0.83589   1.00000   0.91061    117727
           1    0.99796   0.96598   0.98171     98202
           2    0.99882   0.99182   0.99530   2847910
           3    1.00000   1.00000   1.00000    250056

    accuracy                        0.99196   3313895
   macro avg    0.95817   0.98945   0.97191   3313895
weighted avg    0.99309   0.99196   0.99225   3313895

